<a href="https://colab.research.google.com/github/MinamiNaoya/ExperimentTools/blob/main/leash_ecfps_and_light_gbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leash Tutorial - ECFPs and Random Forest
## Introduction

There are many ways to represent molecules for machine learning.

In this tutorial we will go through one of the simplest: ECFPs [[1]](https://pubs.acs.org/doi/10.1021/ci100050t) and Random Forest. This technique is surprisingly powerful, and on previous benchmarks often gets uncomfortably close to the state of the art.

First molecule graphs are broken into bags of subgraphs of varying sizes.

![ecfp featurizing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_generation.png)

Then the bag of subgraphs is hashed into a bit vector

![hashing process (chemaxon)](https://docs.chemaxon.com/display/docs/images/download/attachments/1806333/ecfp_folding.png)

This can be thought of as analogous to the [hashing trick](https://en.wikipedia.org/wiki/Feature_hashing) [[2]](https://alex.smola.org/papers/2009/Weinbergeretal09.pdf) on bag of words for NLP problems, from the days before transformers.

RDKit, an open-source cheminformatics tool, is used for generating ECFP features. It facilitates the creation of hashed bit vectors, streamlining the process. We can install it as follows:

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [2]:
!pip install rdkit

The training set is pretty big, but we can treat the parquet files as databases using duckdb. We will use this to sample down to a smaller dataset for demonstration purposes. Lets install duckdb as well.

In [3]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 54.4 MB/s eta 0:00:00


In [1]:
!pip install kaggle

from google.colab import drive
drive.mount('/content/drive')

import os
import json
f = open("/content/drive/MyDrive/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!kaggle competitions download -c leash-BELKA

100% 4.15G/4.16G [00:49<00:00, 128MB/s]
100% 4.16G/4.16G [00:49<00:00, 89.5MB/s]


In [3]:
!unzip '/content/leash-BELKA.zip'

Archive:  /content/leash-BELKA.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test.parquet            
  inflating: train.csv               
  inflating: train.parquet           


## Data Preparation

The training and testing data paths are defined for the .parquet files. We use duckdb to scan search through the large training sets. Just to get started lets sample out an equal number of positive and negatives.

This query selects an equal number of samples where binds equals 0 (non-binding) and 1 (binding), limited to 30,000 each, to avoid model bias towards a particular class.

トレーニングデータとテストデータのパスは.parquetファイルに対して定義されます。duckdbを使用して、大規模なトレーニングセットをスキャン検索します。まずは、ポジティブとネガティブを同数ずつ抽出します。このクエリでは、モデルが特定のクラスに偏らないように、bindsが0（非結合）と1（結合）の同数のサンプルを、それぞれ30,000個に制限して選択します。

In [4]:
import duckdb
import pandas as pd

train_path = '/content/train.parquet'
test_path = '/content/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 40000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 40000)""").df()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## sEH
エポキシドヒドロラーゼ2はEPHX2遺伝子座にコードされており、そのタンパク質産物は一般に「可溶性エポキシドヒドロラーゼ」、またはsEHと略称される。ヒドロラーゼは特定の化学反応を触媒する酵素であり、EPHX2/sEHもまた特定のリン酸基を加水分解する。EPHX2/sEHは、高血圧と糖尿病進行のための潜在的な薬物標的であり、以前のDELの努力からEPHX2/sEHを阻害する低分子が臨床試験に進んだ。
## BRD4
ブロモドメイン4はBRD4遺伝子座にコードされており、そのタンパク質産物もBRD4と命名されている。ブロモドメインは、DNAが巻き付く核内のタンパク質スプール（ヒストンと呼ばれる）に結合し、近くのDNAが転写される可能性に影響を与え、新しい遺伝子産物を作り出す。ブロモドメインは癌の進行に関与しており、その活性を阻害する薬剤が数多く発見されている。
## ALB
第3の標的である血清アルブミンはALB遺伝子座にコードされており、そのタンパク質産物もALBと命名されている。このタンパク質産物は「ヒト血清アルブミン」を意味するHSAと略されることもある。血液中で最も一般的なタンパク質であるALBは、浸透圧（組織から血管内に体液を戻す）を促進し、多くのリガンド、ホルモン、脂肪酸などを輸送するのに使われる。我々は、Active Motif社から購入したALBを審査した。タンパク質の構造情報を応募に取り入れたい応募者のために、アミノ酸配列はUniProtエントリーP02768の25位から609位、結晶構造はPDBエントリー1AO6、予測構造はAlphaFold2エントリーP02768にあります。リガンドが結合したその他のALB結晶構造はPDBにある。

In [5]:
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,248578768,O=C(O)C[C@@H](Cc1ccc(F)cc1)NC(=O)OCC1c2ccccc2-...,COC(=O)c1cc(N)cs1,Cc1nc(CN)oc1C.Cl.Cl,COC(=O)c1cc(Nc2nc(NCc3nc(C)c(C)o3)nc(N[C@@H](C...,HSA,0
1,148955464,O=C(Nc1c(F)cc(Br)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NC[C@@H]1C[C@@H]2C=C[C@H]1C2,Nc1cc(C(F)(F)F)cnc1Cl,O=C(N[Dy])c1cc(Br)cc(F)c1Nc1nc(NC[C@@H]2C[C@@H...,HSA,0
2,211146662,O=C(Nc1cccc(C(=O)O)c1C(=O)O)OCC1c2ccccc2-c2ccc...,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],C=CCNC(=O)CN.Cl,C=CCNC(=O)CNc1nc(Nc2cc(N3CCNCC3)ccc2[N+](=O)[O...,sEH,0
3,192557727,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,NCc1ccc2[nH]ccc2c1,Cc1cc(N)nn1C,Cc1cc(Nc2nc(NCc3ccc4[nH]ccc4c3)nc(Nc3ccc(C(=O)...,BRD4,0
4,9413451,C=CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,N#CC1=C(N)CCC1,Nc1nc(Br)cn2ccnc12,C=CC[C@@H](Nc1nc(NC2=C(C#N)CCC2)nc(Nc2nc(Br)cn...,BRD4,0
...,...,...,...,...,...,...,...
79995,247648045,O=C(O)C[C@@H](Cc1ccc(Cl)cc1Cl)NC(=O)OCC1c2cccc...,Cc1cc(C#N)c(N)s1,Nc1nc(Cl)c2[nH]cnc2n1,Cc1cc(C#N)c(Nc2nc(Nc3nc(Cl)c4[nH]cnc4n3)nc(N[C...,HSA,1
79996,177453330,O=C(Nc1cc(I)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ccc2c(c1)CNC2=O,Nc1nc2ccccc2s1,O=C1NCc2cc(Nc3nc(Nc4nc5ccccc5s4)nc(Nc4cc(I)ccc...,BRD4,1
79997,81471246,O=C(NC1(C(=O)O)CCc2ccccc21)OCC1c2ccccc2-c2ccccc21,Nc1cc(Cl)c(O)c(Cl)c1,C=CCOC(C)CN,C=CCOC(C)CNc1nc(Nc2cc(Cl)c(O)c(Cl)c2)nc(NC2(C(...,BRD4,1
79998,82769183,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,Nc1ncc([N+](=O)[O-])s1,CC1CCCC(CN)O1,CC1CCCC(CNc2nc(NCC3CCC(C(=O)N[Dy])CC3)nc(Nc3nc...,sEH,1


### PDBファイルのダウンロード

In [6]:
!pip install biopython
import time
import urllib
from Bio.PDB import PDBList

# 7jkz: BED4 1ao6: HSA sEH: 3ily
pdb_ids = ['1ao6', '7jkz', '3i1y']

def download_file(url, dst_path):
    with urllib.request.urlopen(url) as web_file:
        with open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())

# AlphaFold
url = "https://alphafold.ebi.ac.uk/files/AF-P02768-F1-model_v4.pdb"

dst_path = "AF-P02768-F1-model_v4.pdb"
download_file(url, dst_path)

pdbl = PDBList()

for pdb_id in pdb_ids:
    pdbl.retrieve_pdb_file(pdb_id, pdir='pdb_files/')
    time.sleep(10)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.1 MB/s eta 0:00:00


In [7]:
import Bio.PDB
from Bio.PDB import PDBParser
import gzip

parser = PDBParser()

pdb_parser = PDBParser(QUIET=True)
structure = pdb_parser.get_structure('X', '/content/AF-P02768-F1-model_v4.pdb')

for model in structure.get_list():
  for chain in model.get_list():
    print(chain.get_id())
    for residue in chain.get_list():
            print(residue.get_resname(), end = ' ')

A
MET LYS TRP VAL THR PHE ILE SER LEU LEU PHE LEU PHE SER SER ALA TYR SER ARG GLY VAL PHE ARG ARG ASP ALA HIS LYS SER GLU VAL ALA HIS ARG PHE LYS ASP LEU GLY GLU GLU ASN PHE LYS ALA LEU VAL LEU ILE ALA PHE ALA GLN TYR LEU GLN GLN CYS PRO PHE GLU ASP HIS VAL LYS LEU VAL ASN GLU VAL THR GLU PHE ALA LYS THR CYS VAL ALA ASP GLU SER ALA GLU ASN CYS ASP LYS SER LEU HIS THR LEU PHE GLY ASP LYS LEU CYS THR VAL ALA THR LEU ARG GLU THR TYR GLY GLU MET ALA ASP CYS CYS ALA LYS GLN GLU PRO GLU ARG ASN GLU CYS PHE LEU GLN HIS LYS ASP ASP ASN PRO ASN LEU PRO ARG LEU VAL ARG PRO GLU VAL ASP VAL MET CYS THR ALA PHE HIS ASP ASN GLU GLU THR PHE LEU LYS LYS TYR LEU TYR GLU ILE ALA ARG ARG HIS PRO TYR PHE TYR ALA PRO GLU LEU LEU PHE PHE ALA LYS ARG TYR LYS ALA ALA PHE THR GLU CYS CYS GLN ALA ALA ASP LYS ALA ALA CYS LEU LEU PRO LYS LEU ASP GLU LEU ARG ASP GLU GLY LYS ALA SER SER ALA LYS GLN ARG LEU LYS CYS ALA SER LEU GLN LYS PHE GLY GLU ARG ALA PHE LYS ALA TRP ALA VAL ALA ARG LEU SER GLN ARG PHE PRO LYS AL

## Feature Preprocessing

Lets grab the smiles for the fully assembled molecule `molecule_smiles` and generate ecfps for it. We could choose different radiuses or bits, but 2 and 1024 is pretty standard.

In [9]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder


# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
# Define the function to generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    morgan_gen = GetMorganGenerator(radius=radius, fpSize=bits)
    return list(morgan_gen.GetFingerprint(molecule))


df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [10]:
def protein_name_to_pdb(protein_name):
  if protein_name == 'BRD4':
    return '/content/pdb_files/7jkz.cif'
  if protein_name == 'HSA':
    return '/content/pdb_files/1ao6.cif'
  if protein_name == 'sEH':
    return '/content/pdb_files/3i1y.cif'

df['pdb_file_path'] = df['protein_name'].apply(protein_name_to_pdb)

In [11]:
df['pdb_file_path']

0        /content/pdb_files/1ao6.cif
1        /content/pdb_files/1ao6.cif
2        /content/pdb_files/3i1y.cif
3        /content/pdb_files/7jkz.cif
4        /content/pdb_files/7jkz.cif
                    ...             
79995    /content/pdb_files/1ao6.cif
79996    /content/pdb_files/7jkz.cif
79997    /content/pdb_files/7jkz.cif
79998    /content/pdb_files/3i1y.cif
79999    /content/pdb_files/7jkz.cif
Name: pdb_file_path, Length: 80000, dtype: object

In [12]:

from Bio.PDB import MMCIFParser
mmcif_parser = MMCIFParser(QUIET=True)
def get_amino_list_from_pdb(pdb_file_path) -> list:
    structure = mmcif_parser.get_structure('X', pdb_file_path)

    sequences = []
    for model in structure:
        for chain in model:
            seq = ''
            for residue in chain:
                if residue.id[0] == ' ':
                    seq += residue.get_resname()
            sequences.append(seq)
    return sequences

def process_sequence(sequence: list):
    sequence = sequence
    return [sequence[i:i+3] for i in range(0, len(sequence), 3)]
sequences = get_amino_list_from_pdb('/content/pdb_files/1ao6.cif')
processed_sequence = process_sequence(sequences[0])
print(processed_sequence)


['SER', 'GLU', 'VAL', 'ALA', 'HIS', 'ARG', 'PHE', 'LYS', 'ASP', 'LEU', 'GLY', 'GLU', 'GLU', 'ASN', 'PHE', 'LYS', 'ALA', 'LEU', 'VAL', 'LEU', 'ILE', 'ALA', 'PHE', 'ALA', 'GLN', 'TYR', 'LEU', 'GLN', 'GLN', 'CYS', 'PRO', 'PHE', 'GLU', 'ASP', 'HIS', 'VAL', 'LYS', 'LEU', 'VAL', 'ASN', 'GLU', 'VAL', 'THR', 'GLU', 'PHE', 'ALA', 'LYS', 'THR', 'CYS', 'VAL', 'ALA', 'ASP', 'GLU', 'SER', 'ALA', 'GLU', 'ASN', 'CYS', 'ASP', 'LYS', 'SER', 'LEU', 'HIS', 'THR', 'LEU', 'PHE', 'GLY', 'ASP', 'LYS', 'LEU', 'CYS', 'THR', 'VAL', 'ALA', 'THR', 'LEU', 'ARG', 'GLU', 'THR', 'TYR', 'GLY', 'GLU', 'MET', 'ALA', 'ASP', 'CYS', 'CYS', 'ALA', 'LYS', 'GLN', 'GLU', 'PRO', 'GLU', 'ARG', 'ASN', 'GLU', 'CYS', 'PHE', 'LEU', 'GLN', 'HIS', 'LYS', 'ASP', 'ASP', 'ASN', 'PRO', 'ASN', 'LEU', 'PRO', 'ARG', 'LEU', 'VAL', 'ARG', 'PRO', 'GLU', 'VAL', 'ASP', 'VAL', 'MET', 'CYS', 'THR', 'ALA', 'PHE', 'HIS', 'ASP', 'ASN', 'GLU', 'GLU', 'THR', 'PHE', 'LEU', 'LYS', 'LYS', 'TYR', 'LEU', 'TYR', 'GLU', 'ILE', 'ALA', 'ARG', 'ARG', 'HIS', 'PRO'

In [ ]:
#df['amino_seq'] = df['pdb_file_path'].apply(get_amino_list_from_pdb) 計算時間がかかりすぎる。

In [13]:
# 計算量を減らすため

sequences_1ao6 = get_amino_list_from_pdb('/content/pdb_files/1ao6.cif')
processed_sequence_1ao6 = process_sequence(sequences_1ao6[0])

sequences_3i1y = get_amino_list_from_pdb('/content/pdb_files/3i1y.cif')
processed_sequence_3i1y = process_sequence(sequences_3i1y[0])

sequences_7jkz = get_amino_list_from_pdb('/content/pdb_files/7jkz.cif')
processed_sequence_7jkz = process_sequence(sequences_7jkz[0])
# 7jkz: BRD4 1ao6: HSA sEH: 3ily
pdb_dict = {
    'HSA': processed_sequence_1ao6,
    'sEH': processed_sequence_3i1y,
    'BRD4': processed_sequence_7jkz
}
pdb_dict_amino = {
    'HSA': sequences_1ao6,
    'sEH': sequences_3i1y,
    'BRD4': sequences_7jkz
}


## DTIについて(Drug-Target interaction)
https://www.nature.com/articles/s41598-023-30026-y

In [14]:
df['amino_seq_list'] = df['protein_name'].apply(lambda x: pdb_dict.get(x, x))

In [15]:
df['amino_seq'] = df['protein_name'].apply(lambda x: pdb_dict_amino.get(x, x))

In [16]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp,pdb_file_path,amino_seq_list,amino_seq
0,248578768,O=C(O)C[C@@H](Cc1ccc(F)cc1)NC(=O)OCC1c2ccccc2-...,COC(=O)c1cc(N)cs1,Cc1nc(CN)oc1C.Cl.Cl,COC(=O)c1cc(Nc2nc(NCc3nc(C)c(C)o3)nc(N[C@@H](C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e490>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/1ao6.cif,"[SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...",[SERGLUVALALAHISARGPHELYSASPLEUGLYGLUGLUASNPHE...
1,148955464,O=C(Nc1c(F)cc(Br)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NC[C@@H]1C[C@@H]2C=C[C@H]1C2,Nc1cc(C(F)(F)F)cnc1Cl,O=C(N[Dy])c1cc(Br)cc(F)c1Nc1nc(NC[C@@H]2C[C@@H...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e500>,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/1ao6.cif,"[SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...",[SERGLUVALALAHISARGPHELYSASPLEUGLYGLUGLUASNPHE...
2,211146662,O=C(Nc1cccc(C(=O)O)c1C(=O)O)OCC1c2ccccc2-c2ccc...,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],C=CCNC(=O)CN.Cl,C=CCNC(=O)CNc1nc(Nc2cc(N3CCNCC3)ccc2[N+](=O)[O...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e2d0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/3i1y.cif,"[ARG, ALA, ALA, VAL, PHE, ASP, LEU, ASP, GLY, ...",[ARGALAALAVALPHEASPLEUASPGLYVALLEUALALEUPROALA...
3,192557727,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,NCc1ccc2[nH]ccc2c1,Cc1cc(N)nn1C,Cc1cc(Nc2nc(NCc3ccc4[nH]ccc4c3)nc(Nc3ccc(C(=O)...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e3b0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...
4,9413451,C=CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,N#CC1=C(N)CCC1,Nc1nc(Br)cn2ccnc12,C=CC[C@@H](Nc1nc(NC2=C(C#N)CCC2)nc(Nc2nc(Br)cn...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e340>,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...


In [17]:
df['amino_seq_list']

0        [SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...
1        [SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...
2        [ARG, ALA, ALA, VAL, PHE, ASP, LEU, ASP, GLY, ...
3        [LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...
4        [LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...
                               ...                        
79995    [SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...
79996    [LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...
79997    [LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...
79998    [ARG, ALA, ALA, VAL, PHE, ASP, LEU, ASP, GLY, ...
79999    [LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...
Name: amino_seq_list, Length: 80000, dtype: object

In [18]:
df['amino_seq_str'] = df['amino_seq_list'].apply(lambda x: ','.join(x))

## Train Model

In [19]:
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))
amino_onehot = onehot_encoder.fit_transform(df['amino_seq_str'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded amino seq
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), amino_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")




Mean Average Precision (mAP): 0.96


### LightGBM

In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import lightgbm as lgb
import pandas as pd

# One-hot encode the protein_name and amino_seq_str
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))
amino_onehot = onehot_encoder.fit_transform(df['amino_seq_str'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded amino seq
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), amino_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the LightGBM model
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]  # Probability of the positive class

# Calculate the mean average precision
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")


[LightGBM] [Info] Number of positive: 31979, number of negative: 32021
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.027679 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2016
[LightGBM] [Info] Number of data points in the train set: 64000, number of used features: 1008
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499672 -> initscore=-0.001313
[LightGBM] [Info] Start training from score -0.001313
Mean Average Precision (mAP): 0.96


Look at that Average Precision score. We did amazing!

Actually no, we just overfit. This is likely recurring theme for this competition. It is easy to predict molecules that come from the same corner of chemical space, but generalizing to new molecules is extremely difficult.

In [21]:
import gc
gc.collect()

35

In [22]:
df

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,molecule,ecfp,pdb_file_path,amino_seq_list,amino_seq,amino_seq_str
0,248578768,O=C(O)C[C@@H](Cc1ccc(F)cc1)NC(=O)OCC1c2ccccc2-...,COC(=O)c1cc(N)cs1,Cc1nc(CN)oc1C.Cl.Cl,COC(=O)c1cc(Nc2nc(NCc3nc(C)c(C)o3)nc(N[C@@H](C...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e490>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/1ao6.cif,"[SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...",[SERGLUVALALAHISARGPHELYSASPLEUGLYGLUGLUASNPHE...,"SER,GLU,VAL,ALA,HIS,ARG,PHE,LYS,ASP,LEU,GLY,GL..."
1,148955464,O=C(Nc1c(F)cc(Br)cc1C(=O)O)OCC1c2ccccc2-c2ccccc21,NC[C@@H]1C[C@@H]2C=C[C@H]1C2,Nc1cc(C(F)(F)F)cnc1Cl,O=C(N[Dy])c1cc(Br)cc(F)c1Nc1nc(NC[C@@H]2C[C@@H...,HSA,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e500>,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/1ao6.cif,"[SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...",[SERGLUVALALAHISARGPHELYSASPLEUGLYGLUGLUASNPHE...,"SER,GLU,VAL,ALA,HIS,ARG,PHE,LYS,ASP,LEU,GLY,GL..."
2,211146662,O=C(Nc1cccc(C(=O)O)c1C(=O)O)OCC1c2ccccc2-c2ccc...,Nc1cc(N2CCNCC2)ccc1[N+](=O)[O-],C=CCNC(=O)CN.Cl,C=CCNC(=O)CNc1nc(Nc2cc(N3CCNCC3)ccc2[N+](=O)[O...,sEH,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e2d0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/3i1y.cif,"[ARG, ALA, ALA, VAL, PHE, ASP, LEU, ASP, GLY, ...",[ARGALAALAVALPHEASPLEUASPGLYVALLEUALALEUPROALA...,"ARG,ALA,ALA,VAL,PHE,ASP,LEU,ASP,GLY,VAL,LEU,AL..."
3,192557727,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,NCc1ccc2[nH]ccc2c1,Cc1cc(N)nn1C,Cc1cc(Nc2nc(NCc3ccc4[nH]ccc4c3)nc(Nc3ccc(C(=O)...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e3b0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...,"LYS,VAL,SER,GLU,GLN,LEU,LYS,CYS,CYS,SER,GLY,IL..."
4,9413451,C=CC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,N#CC1=C(N)CCC1,Nc1nc(Br)cn2ccnc12,C=CC[C@@H](Nc1nc(NC2=C(C#N)CCC2)nc(Nc2nc(Br)cn...,BRD4,0,<rdkit.Chem.rdchem.Mol object at 0x7acfc906e340>,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...,"LYS,VAL,SER,GLU,GLN,LEU,LYS,CYS,CYS,SER,GLY,IL..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,247648045,O=C(O)C[C@@H](Cc1ccc(Cl)cc1Cl)NC(=O)OCC1c2cccc...,Cc1cc(C#N)c(N)s1,Nc1nc(Cl)c2[nH]cnc2n1,Cc1cc(C#N)c(Nc2nc(Nc3nc(Cl)c4[nH]cnc4n3)nc(N[C...,HSA,1,<rdkit.Chem.rdchem.Mol object at 0x7acfc8748c10>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/1ao6.cif,"[SER, GLU, VAL, ALA, HIS, ARG, PHE, LYS, ASP, ...",[SERGLUVALALAHISARGPHELYSASPLEUGLYGLUGLUASNPHE...,"SER,GLU,VAL,ALA,HIS,ARG,PHE,LYS,ASP,LEU,GLY,GL..."
79996,177453330,O=C(Nc1cc(I)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,Nc1ccc2c(c1)CNC2=O,Nc1nc2ccccc2s1,O=C1NCc2cc(Nc3nc(Nc4nc5ccccc5s4)nc(Nc4cc(I)ccc...,BRD4,1,<rdkit.Chem.rdchem.Mol object at 0x7acfc8748c80>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...,"LYS,VAL,SER,GLU,GLN,LEU,LYS,CYS,CYS,SER,GLY,IL..."
79997,81471246,O=C(NC1(C(=O)O)CCc2ccccc21)OCC1c2ccccc2-c2ccccc21,Nc1cc(Cl)c(O)c(Cl)c1,C=CCOC(C)CN,C=CCOC(C)CNc1nc(Nc2cc(Cl)c(O)c(Cl)c2)nc(NC2(C(...,BRD4,1,<rdkit.Chem.rdchem.Mol object at 0x7acfc8748cf0>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",/content/pdb_files/7jkz.cif,"[LYS, VAL, SER, GLU, GLN, LEU, LYS, CYS, CYS, ...",[LYSVALSERGLUGLNLEULYSCYSCYSSERGLYILELEULYSGLU...,"LYS,VAL,SER,GLU,GLN,LEU,LYS,CYS,CYS,SER,GLY,IL..."
79998,82769183,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,Nc1ncc([N+](=O)[O-])s1,CC1CCCC(CN)O1,CC1CCCC(CNc2nc(NCC3CCC(C(=O)N[Dy])CC3)nc(Nc3nc...,sEH,1,<rdkit.Chem.rdchem.Mol object at 0x7acfc8748d60>,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

## Test Prediction

 The trained Random Forest model is then used to predict the binding probabilities. These predictions are saved to a CSV file, which serves as the submission file for the Kaggle competition.

In [23]:
"""
import os
import gc
import pandas as pd
# Process the test.parquet file chunk by chunk
test_file = '/content/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=10000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['amino_seq_list'] = df_test['protein_name'].apply(lambda x: pdb_dict.get(x, x))
    df_test['amino_seq_str'] = df_test['amino_seq_list'].apply(lambda x: ','.join(x))
    # One-hot encode the protein_name, amino_acid
    #protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    amino_onehot = onehot_encoder.fit_transform(df_test['amino_seq_str'].values.reshape(-1, 1))
    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + amino_acid for ecfp, amino_acid in zip(df_test['ecfp'].tolist(), amino_onehot.tolist())]

    # Predict the probabilities
    probabilities = rf_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
    gc.collect()
"""

KeyboardInterrupt: 

In [24]:
import os
import gc
import pandas as pd
# Process the test.parquet file chunk by chunk
test_file = '/content/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file

# Read the test.parquet file into a pandas DataFrame
for df_test in pd.read_csv(test_file, chunksize=10000):

    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)
    df_test['amino_seq_list'] = df_test['protein_name'].apply(lambda x: pdb_dict.get(x, x))
    df_test['amino_seq_str'] = df_test['amino_seq_list'].apply(lambda x: ','.join(x))
    # One-hot encode the protein_name, amino_acid
    #protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))
    amino_onehot = onehot_encoder.fit_transform(df_test['amino_seq_str'].values.reshape(-1, 1))
    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + amino_acid for ecfp, amino_acid in zip(df_test['ecfp'].tolist(), amino_onehot.tolist())]

    # Predict the probabilities
    probabilities = lgb_model.predict_proba(X_test)[:, 1]

    # Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': probabilities})

    # Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))
    gc.collect()

In [25]:
import pandas as pd

df1 = pd.read_csv("submission.csv")


## 方法
1. ChemBertaでSmilesを特徴量にしてみる。
2. ランダムフォレスト以外のモデルを利用してみる。
3. アミノ酸配列、DNA配列の情報から特徴量を生成する。


In [28]:
df1.head()

,id,binds
0,295246830,0.29
1,295246831,0.21
2,295246832,0.18
3,295246833,0.34
4,295246834,0.39


In [29]:
df1 = df1.drop_duplicates(subset=['id'])